In [7]:
import quartz
import torch
import dgl
import time
from icecream import ic

In [2]:
gate_set = ['h', 'cx', 't', 'tdg']
ecc_file = 'bfs_verified_simplified.json'
no_increase = False
include_nop = True
quartz_context = quartz.QuartzContext(
    gate_set=gate_set,
    filename=ecc_file,
    # TODO  we need to include xfers that lead to gate increase when training?
    # we may exclude them when generating the dataset for pre-training
    # TODO  to make the task easier, we exclude those xfers currently
    no_increase=no_increase,
    include_nop=include_nop,
)
num_xfers = quartz_context.num_xfers
parser = quartz.PyQASMParser(context=quartz_context)
ic(num_xfers)


ic| num_xfers: 3904


3904

In [22]:
qasm_str = 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\nh q[4];\ncx q[3],q[4];\ntdg q[4];\ncx q[2],q[4];\nt q[4];\ncx q[3],q[4];\ntdg q[4];\ncx q[3],q[2];\ntdg q[2];\ncx q[3],q[2];\nt q[3];\ncx q[2],q[4];\nh q[3];\nt q[4];\ncx q[1],q[3];\ntdg q[3];\ncx q[0],q[3];\nt q[3];\ncx q[1],q[3];\ntdg q[3];\ncx q[0],q[3];\ncx q[0],q[1];\nt q[3];\ntdg q[1];\nh q[3];\ncx q[0],q[1];\ncx q[3],q[4];\nt q[0];\nt q[1];\nt q[4];\ncx q[2],q[4];\ntdg q[4];\ncx q[3],q[4];\nt q[4];\ncx q[2],q[4];\ntdg q[4];\ncx q[2],q[3];\nh q[4];\nt q[3];\ncx q[2],q[3];\ntdg q[3];\nh q[3];\ncx q[1],q[3];\nt q[3];\ncx q[0],q[3];\ntdg q[3];\ncx q[1],q[3];\nt q[3];\ncx q[0],q[3];\ncx q[0],q[1];\ntdg q[3];\nt q[1];\nh q[3];\ncx q[0],q[1];\ntdg q[0];\ntdg q[1];\n'
s_time = time.time_ns()
dag = parser.load_qasm_str(qasm_str)
graph = quartz.PyGraph(context=quartz_context, dag=dag)
e_time = time.time_ns()
print(f'{(e_time - s_time) / 1e6} ms')

3.75787 ms


In [14]:
s_time = time.time_ns()
graph2 = quartz.PyGraph.from_qasm_str(context=quartz_context, qasm_str=qasm_str)
e_time = time.time_ns()
print(f'{(e_time - s_time) / 1e6} ms')

3.339538 ms


In [6]:
while True:
    avxfers = []
    for i in range(graph.num_nodes):
        avxfers.append(
            graph.available_xfers(
                context=quartz_context,
                node=graph.get_node_from_id(id=i)
            )
        )
    avxfers2 = []
    for i in range(graph2.num_nodes):
        avxfers2.append(
            graph2.available_xfers(
                context=quartz_context,
                node=graph2.get_node_from_id(id=i)
            )
        )
    print(avxfers == avxfers2)

True
True
True
True
True


KeyboardInterrupt: 

In [ ]:
int(torch.multinomial(torch.Tensor([1,2,3,4]), 1))

In [ ]:
next_graph, next_nodes = \
    graph.apply_xfer_with_local_state_tracking(
        xfer=quartz_context.get_xfer_from_id(id=action.xfer),
        node=graph.get_node_from_id(id=action.node)
    )

In [ ]:
avxfers = []
for i in range(graph.num_nodes):
    avxfers.append(
        graph.available_xfers(
            context=quartz_context,
            node=graph.get_node_from_id(id=i)
        )
    )
avxfers

In [ ]:
qasm2 = graph.to_qasm_str()
g2 = quartz.PyGraph(context=quartz_context, dag=parser.load_qasm_str(qasm2))
avxfers2 = []
for i in range(g2.num_nodes):
    avxfers2.append(
        g2.available_xfers(
            context=quartz_context,
            node=g2.get_node_from_id(id=i)
        )
    )
avxfers2 == avxfers

In [ ]:
max([])